In [ ]:
!pip install scikit-image 

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd

In [2]:
image_dataset=pd.DataFrame()
img_path="C:/Users/Bumble Bee/Downloads/Research/Oral Cancer Detection/ftmp4cvtmb-1/First Set/Selected_Oscc/OSCC-1 - export/New folder/Image/"
for image in os.listdir(img_path):
    df= pd.DataFrame()
    input_img = cv2.imread(img_path+ image)
    if input_img.ndim == 3 and input_img.shape[-1]==3:
        img=cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
    elif input_img.ndim==2:
        img=input_img
    else:
        raise Exception("The module works on gray")
            
    pixel_value=img.reshape(-1)
    df["pixel_value"]= pixel_value
    df["image_name"]=image
    num=1
    kernels=[]
    for theta in range(2):
        theta=theta /4 *np.pi
        for sigma in (1,3):
                for lamda in np.arange(0,np.pi,np.pi /4):
                    for gamma in(0.05, 0.5):
                        gabor_label='Gabor'+str(num)
                        ksize=9
                        kernel=cv2.getGaborKernel((ksize,ksize),sigma,theta,lamda,gamma,0, ktype=cv2.CV_32F)
                        kernels.append(kernel)
                        fimg=cv2.filter2D(img,cv2.CV_8UC3,kernel)
                        filtered_img=fimg.reshape(-1)
                        df[gabor_label]=filtered_img
                        num+=1
                        
    edges=cv2.Canny(img,100,200)
    edges1=edges.reshape(-1)
    df['Canny edge']=edges1
        
    edge_roberts=roberts(img)
    edge_roberts1=edge_roberts.reshape(-1)
    df['Roberts']=edge_roberts1

    edge_sobel =sobel(img)
    edge_sobel1=edge_sobel.reshape(-1)
    df['Sobel']= edge_sobel1

    edge_scharr = scharr(img)
    edge_scharr1 =edge_scharr.reshape(-1)
    df['Scharr']=edge_scharr1

    edge_prewitt= prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt']=edge_prewitt1

    gaussian_img = nd.gaussian_filter(img,sigma=3)
    gaussian_img1= gaussian_img.reshape(-1)
    df['Gaussian s3']= gaussian_img1

    gaussian_img2= nd.gaussian_filter(img,sigma=7)
    gaussian_img3= gaussian_img.reshape(-1)
    df['Gaussian s7']=gaussian_img3

    median_img= nd.median_filter(img,size=3)
    median_img1= median_img.reshape(-1)
    df['Median s3']=median_img1
        
    image_dataset=pd.concat([image_dataset,df],ignore_index=True)

In [3]:
mask_dataset =pd.DataFrame()
mask_path ="C:/Users/Bumble Bee/Downloads/Research/Oral Cancer Detection/ftmp4cvtmb-1/First Set/Selected_Oscc/OSCC-1 - export/New folder/mask/"

for mask in os.listdir(mask_path):
    df2= pd.DataFrame()
    masked_img= cv2.imread(mask_path+ mask)
    masked_img=cv2.cvtColor(masked_img,cv2.COLOR_BGR2GRAY)
    label_value=masked_img.reshape(-1)
    df2["Label_value"]= label_value
    df2["Mask_name"]=mask
    
    mask_dataset = pd.concat([mask_dataset,df2],ignore_index=True)
    

In [4]:
dataset=pd.concat([df, mask_dataset],axis=1)
dataset.fillna(0, inplace=True)
dataset =dataset[dataset.Label_value !=0]

In [5]:
x=dataset.drop(labels =["image_name","Mask_name","Label_value"],axis=1)
y=dataset["Label_value"].values

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train , y_test = train_test_split(x,y,test_size=0.2, random_state=20)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(x_train,y_train)

In [ ]:
import pickle
filename = "oscc_model1"
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def feature_extraction(img):
    tmp_df = pd.DataFrame()

    img2 = img.reshape(-1)
    tmp_df['Original Image'] = img2

    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                
                    gabor_label = 'Gabor' + str(num)
                    ksize=9
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    tmp_df[gabor_label] = filtered_img 
                    num += 1
                    
    edges = cv2.Canny(img, 100,200)   
    edges1 = edges.reshape(-1)
    tmp_df['Canny Edge'] = edges1 

    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    tmp_df['Roberts'] = edge_roberts1

    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    tmp_df['Sobel'] = edge_sobel1

    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    tmp_df['Scharr'] = edge_scharr1

    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    tmp_df['Prewitt'] = edge_prewitt1

    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    tmp_df['Gaussian s3'] = gaussian_img1

    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    tmp_df['Gaussian s7'] = gaussian_img3

    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    tmp_df['Median s3'] = median_img1

    return tmp_df

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import glob
import matplotlib.pyplot as plt 

path = "C:/Users/Bumble Bee/Downloads/Research/Oral Cancer Detection/ftmp4cvtmb-1/First Set/Selected_Oscc/Basic Segmentation/*.jpg"
for file in glob.glob(path):
    print(file)
    img1 = cv2.imread(file)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    X = feature_extraction(img)
    result = loaded_model.predict(X)
    segmented = result.reshape((img.shape))
    
    name = os.path.splitext(os.path.basename(file))[0]
    output_dir = "E:/Segmented"
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, name + '_segmented.jpg')
    plt.imsave(output_file, segmented, cmap='jet')


In [7]:
from sklearn.svm import SVC
model_SVM =SVC(kernel='rbf')
model_SVM.fit(x_train,y_train)

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
from sklearn import metrics
prediction_test_random= model.predict(x_test)
prediction_test_svm =model_SVM.predict(x_test)
print("Accuracy= ",metrics.accuracy_score(y_test,prediction_test_random))
print("Accuracy= ",metrics.accuracy_score(y_test,prediction_test_svm))

In [ ]:
labeled_img=cv2.imread()

In [ ]:
labeled_img=cv2.cvtColor(labeled_img,cv2.COLOR_BGR2GRAY)
labeled_img1=labeled_img.reshape(-1)
df['Labels']=labeled_img1
printf(df.head())

In [ ]:
original_img_data =df.drop(labels =['Labels'],axis=1)
df=df[df.Labels !=0]

In [ ]:
y= df['Label'].values
from sklearn.preprocessing import LabelEncoder
y=LabelEncoder().fit_transfrom(y)

In [ ]:
feature_list=list(x.columns)
feauture_imp=pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending)
print(feature_imp)

In [ ]:
from sklearn.svm import LinearSVC
model_SVM =LinearSVC(max_iter=100)
model_SVM.fit(x_train,y_train)

In [ ]:
prediction_random= model.predict(x_test)
prediction_SVM = model_SVM.predict(x_test)